# Assignment 3: Wallapop Webscraper

## Group A2

### Alejandra Saldivar, Anya Du Plessis, Lennart Heinacher, Henrique Villarinho & Kaustubh Singh

<div class="alert alert-success">To run this code successfully please install Selenium, BeautifulSoup4 & Numpy</div>

### Please use the following code in your terminal to install the respective packages

<b>pip install selenium</b>

<b>pip install beautifulsoup4</b>

<b>pip install numpy</b>


In [1]:
import selenium
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import requests
import bs4
import numpy as np
import pandas as pd

import datetime
import locale

In [2]:
#Launch Chrome driver 
path = 'chromedriver.exe'
driver = webdriver.Chrome(path)
driver.maximize_window()

/var/folders/89/br65hwbs3wx_qvg3y8l8n10c0000gp/T/ipykernel_60828/628060905.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [3]:
#Enter to Wallopp
time.sleep(1)
driver.get('https://es.wallapop.com')

In [4]:
#Accept cookies
time.sleep(1)
WebDriverWait(driver, timeout= 15).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="onetrust-accept-btn-handler"]'))).click()

In [5]:
#Find the searching box and search for 'Bicicletas'
search = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='keywords']")))
search.send_keys('Bicicleta')

In [6]:
#Identify the whole list of results and then select the results under category 'Bicicletas'
list_res = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '//ul[@class="Search__list"]')))
list_res.find_element(By.XPATH, "//span[@class='Search__suggestion-category' and text()='Bicicletas']").click()

## We will now filter the search according to the criteria proicded

**Location:** Setting location as **“España, Barcelona”** and narrow down the search to a **maximum of “10km”**

**Price:** Limitting the price to **800€.**

**Subfield:** Within the “Bicicletas” field, there are multiple subfields available. Narrow down the search to include only results in the **“Bicicletas y triciclos”** subfield. 

**Labels:** You can further narrow down your search by choosing more refined subfields. You should retrieve only those results that correspond to **“Bicicletas de carretera” (road bikes), “Bicicletas plegables” (foldable bikes) or “MTB” (mountain bikes).**

**State:** Only include those results that correspond to bikes that are **“Nuevo” (New), “Como nuevo” (As good as new) and “En buen estado” (In good condition).**

In [7]:
# Function to click in Filters button

def click_filter():
    filters = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='Bubble__content' and text()='Filtros']"))).click()

In [8]:
click_filter()

### Location: 
Setting location as **“España, Barcelona”** and narrow down the search to a **maximum of “10km”**


In [9]:
#From the filters menu identify the location filter 
loc_subfilter = driver.find_element(By.XPATH, '//div[@class="FilterGroup__filter m-1 ng-star-inserted"]')
location_filter = loc_subfilter.find_element(By.XPATH,"//span[@class='SelectOption__label text-truncate overflow-hidden' and text()= 'España, Madrid']").click()
location = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/div[1]/input')))

#Filter for 'Barcelona'
location.clear()
location.send_keys('Barcelona')

#Select the first result 'España, Barcelona' 
location_result = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='SearchBox__suggestion d-flex flex-column justify-content-center ng-star-inserted']"))).click()

#Identify the slider and move it to 10K
slider = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-slider-form/form/ngx-slider/span[5]')))
move = ActionChains(driver)

move.click_and_hold(slider).move_by_offset(-300,100).release().perform()

#Click "Aplicar"
driver.find_element(By.XPATH, "//button[@class='btn btn--width btn-filter btn-primary']").click()

### Price:
Limitting the price to **800€.**

In [10]:
#Identify the filter category and filter by '800 euros'
price_filter = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-range-filter/tsl-filter-template/div/form/div/div[2]/input')))
price_filter.clear()
price_filter.send_keys('800')

### Subfield: 
Within the “Bicicletas” field, there are multiple subfields available. Narrow down the search to include only results in the **“Bicicletas y triciclos”** subfield. 

In [11]:
# Function to click on Subfield tab in the filter menu 
# Within the “Bicicletas” field, there are multiple subfields available. Click in the subcategory criteria
def subfield_select():
    
    sel_sub = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/tsl-select-parent-option/div/tsl-select-option/div/div/span'))).click()
    time.sleep(2)

    #Click in the filter 'Bicicletas y triciclos'
    second_spot = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-multi-select-form/div[3]/tsl-multi-select-option/div/div/p')))
    third_spot = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-multi-select-form/div[2]/tsl-multi-select-option/div/div/p')))

    #As the filter moves between the second and third row we make sure it click in the correct one
    if 'Bicicletas y triciclos' in second_spot.text.strip(): 
        second_spot.click()
    elif 'Bicicletas y triciclos' in third_spot.text.strip(): 
        third_spot.click()

In [12]:
# Function to Go back to the filter menu

def back_filter_type():
    back = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[1]/tsl-svg-icon'))).click()                                                         

In [13]:
# Apply all the filters

def apply_final_filter():
    apply_all = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[2]/tsl-button[2]/button'))).click()
    time.sleep(1)

### Labels: 
You can further narrow down your search by choosing more refined subfields. You should retrieve only those results that correspond to **“Bicicletas de carretera” (road bikes), “Bicicletas plegables” (foldable bikes) or “MTB” (mountain bikes).**

In [14]:
# Function to click on Labels tab in the filter menu 

def select_type(unselect_x, select_y):
    for result in driver.find_elements(By.XPATH, "//*[@class = 'w-100 ng-star-inserted']"):
        if unselect_x in result.text:
            result.find_element(By.XPATH, './/tsl-checkbox-form').click()
        elif select_y in result.text:
            result.find_element(By.XPATH, './/tsl-checkbox-form').click()

### State: 
Only include those results that correspond to bikes that are **“Nuevo” (New), “Como nuevo” (As good as new) and “En buen estado” (In good condition).**

In this step we are scrolling down to get at least 250 posts loaded or all available posts

In [15]:
#Try to find the next page button or scroll down to get more posts
def scroll_down():
    prev_len = -1
    

    while True:

        try:
            WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,"//button[contains(text(),'Ver más productos')]"))).click()

        except:
            driver.execute_script("window.scrollBy(0,document.body.scrollHeight);")
            WebDriverWait(driver, 4)
        curr_len = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))
        if  curr_len>= 250 or prev_len == curr_len:
            break

        prev_len = curr_len
    t_limit = curr_len

In [16]:
# Function to click on the state filter 

def state_filter():

    state_filter = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[4]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/tsl-select-parent-option/div/tsl-select-option/div/div/span'))).click()

In [17]:
# Go back to the filter menu from state

def back_filter_state():
    back2 = driver.find_element(By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[4]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[1]/tsl-svg-icon').click()

Once the posts are loaded we need to go back to the top of the page to start retrieving information

In [18]:
#Function to scroll to the top of the page
def scroll_to_top():
    driver.execute_script("window.scrollTo(0, 220)")

In this step we are visiting the posts one by one and retrieveing all the desired information.

1. “Type” to refere whether the  record corresponds to a “Bicicleta de carretera”, a “Bicicleta plegable” or “MTB”

2. “State” to specify whether the considered bike is “Nuevo”, “Como nuevo” or “En buen estado”.

3. “Children” to identify whether the considered entry corresponds to a children bike or not, identified by the keywords “niño/a”, “niño”, “niña”, “niños” , “niñas” or "niño/as".

4. “Size” (in int form) with the size of the bike which are usually along the ranges 54 - 60 approx.

5. Sometimes the size will be specified as S, M or L, When this happens, store the corresponding information in a new column called "Size (letter)"

6. “Date” (in datetime form) wiith information regarding the date of publication of the post. 

In [20]:
#Creating empty dataframe to store values
df_main = pd.DataFrame(columns = ['Link','Titles', 'Description', 'Price', 'Image', 'Type', 'State', 'Children', 'Size', 'Size_letter','Date']) 

In this step we are changing the prices retrieved from string to float

In [21]:
def prices_correct(prices):

    price_float = []
    correct_prices = []

    for x in range(len(prices)):
        correct_prices.append(prices[x].replace(',', '.'))
        price_float.append(float(correct_prices[x].split('EUR')[0]))
        
    return price_float

In this steps we are looking in the descriptions retrieved for the sizes of the bikes

In [22]:
def bike_size_all(description):

    all_info = []

    for i in range(len(description)):
        if 'talla' in description[i]:
            ls = description[i].split(" ")
            try:
                if len(ls[int(ls.index("talla")) + 1]) < 4:
                    all_info.append(ls[int(ls.index("talla")) + 1])
                else:
                    all_info.append(np.nan)
            except:
                all_info.append(np.nan)
        elif 'talla:' in description[i]:
            try:
                if len(ls[int(ls.index("talla:")) + 1]) < 4:
                    all_info.append(ls[int(ls.index("talla:")) + 1])
                else:
                    all_info.append(np.nan)
            except:
                all_info.append(np.nan)
        else:
            all_info.append(np.nan)
    
    #In this step we are seperate the numbers and strings from the retrieved sizes 

    size_number = []
    size_letter = []


    for i in range(len(all_info)):
        try:
            if len(all_info[i].split(".")) > 1:
                all_info[i] = int(all_info[i].split(".")[0])

            elif len(all_info[i].split(",")) > 1:
                all_info[i] = int(all_info[i].split(",")[0])

            elif len(all_info[i].split("/")) > 1:
                all_info[i] = all_info[i].split("/")[0]

            else:
                all_info[i] = all_info[i]

        except:
            all_info[i] = all_info[i]

    for item in all_info:

        if isinstance(item, int):
            size_number.append(int(item))
            size_letter.append(np.nan)
        elif isinstance(item, float):
            size_number.append(item)
            size_letter.append(np.nan)
        else:
            if item.isdigit():
                size_number.append(int(item))
                size_letter.append(np.nan)
            else:
                size_number.append(np.nan)
                size_letter.append(item)
        
            
    return size_letter, size_number

In this step we are changing the format of the date retrieved from the posts

In [23]:
def date_format(date):
    ls_date = []
    locale.setlocale(locale.LC_TIME, "es_ES.UTF-8")

    for d in range(len(date)):
        try:
            x = datetime.datetime.strptime(date[d], "%d-%b-%Y")
            ls_date.append(pd.to_datetime(x, format = "%d/%m/%Y"))
        except:
            ls_date.append(np.nan)
            
    return ls_date

In this step we are looking in all the titles and descriptions obtained if the bicycle is for kids.

In [24]:
def is_for_kids(description, titles):
    children = []
    kids_check = ['niño/a', 'niño', 'niña', 'niños', 'Niño/a', 'Niño', 'Niña', 'Niños']


    for i in range(len(description)):
        if any(x in titles[i] for x in kids_check):
            children.append(bool(True))
        elif any(x in description[i] for x in kids_check):
            children.append(bool(True))
        else:
            children.append(bool(False))

    return children

In this step we are creating the df with all the information retrieved for 250 posts

In [25]:

def web_scarping(target_limit):

    post_url = []
    description = []
    prices = []
    image_urls = []
    titles = []
    bike_type = []
    state = []
    date = []

    for result in driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'):

        image_urls.append(result.find_element(By.TAG_NAME, "img").get_attribute("src"))
        titles.append(result.find_element(By.TAG_NAME, "img").get_attribute("alt"))
        time.sleep(1)
        
        try:
            result.find_element(By.TAG_NAME, "img").click()
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[-1])
            post_url.append(driver.current_url)
            prices.append(driver.find_element(By.XPATH, '//div[@class="card-product-price-info"]').text)
            description.append(driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[1]/p').text)
            bike_type.append(driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[1]/div[2]/div/div[2]/span').text)
            state.append(driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[1]/div[2]/div[1]/span').text)
            date.append(driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[2]/div[2]/div[1]').text)
            
        except:
            del image_urls[-1]
            del titles[-1]
            
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        if len(post_url) == target_limit:
            break
   
    children_ls = is_for_kids(description, titles)

    prices_final = prices_correct(prices)

    size_string, size_num = bike_size_all(description)

    correct_date = date_format(date)

    df = pd.DataFrame(data= {'Link': post_url[0:target_limit],
                             'Titles': titles[0:target_limit],
                             'Description': description[0:target_limit],
                             'Price': prices_final[0:target_limit],
                             'Image' : image_urls[0:target_limit],
                             'Type': bike_type[0:target_limit],
                             'State': state[0:target_limit],
                             'Children': children_ls[0:target_limit],
                             'Size': size_num[0:target_limit],
                             'Size_letter': size_string[0:target_limit],
                             'Date': correct_date[0:target_limit]
                            }) 

    #dropping duplicated posts
    df2 = df.drop_duplicates()
    
    return df2


<div class="alert alert-success"> By changing value of t_limit below, you can change the limit of number of postings you wish to retrieve from each type & state combination</div>

Combination of **'Bicicletas de carretera' & 'Nuevo'**

In [26]:
# Function to select the Subfield
subfield_select()

In [27]:
for result in driver.find_elements(By.XPATH, "//*[@class = 'w-100 ng-star-inserted']"):
    if 'Bicicletas de carretera' in result.text:
        result.find_element(By.XPATH, './/tsl-checkbox-form').click()

In [28]:
# Go back
back_filter_type()

# Open the state filter tab
state_filter()

In [29]:
for result in driver.find_elements(By.XPATH, "//*[@class = 'w-100 ng-star-inserted']"):
    if 'Nuevo' in result.text:
        result.find_element(By.XPATH, './/tsl-checkbox-form').click()

In [30]:
# Go back
back_filter_state()

# Apply all the filters
apply_final_filter()

In [31]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [32]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

36


In [33]:
# Web scraping from listing
df_1 = web_scarping(posts_num)



df_main = pd.concat([df_main, df_1], axis = 0)

In [34]:
df_main

,Link,Titles,Description,Price,Image,Type,State,Children,Size,Size_letter,Date
0,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña,Bicicleta de montaña para niño/a a partir de 1...,90.0,https://cdn.wallapop.com/images/10420/e5/qy/__...,Bicicletas de carretera,Nuevo,True,NaN,NaN,2022-12-08
1,https://es.wallapop.com/item/bicicleta-deporti...,bicicleta deportiva 290€,No la he usado esta nueva,290.0,https://cdn.wallapop.com/images/10420/e5/q0/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-08
2,https://es.wallapop.com/item/bicicleta-krn-kra...,Bicicleta KRN Krampus Monster Truck,Cuadro de Aluminio Horquilla de Aluminio Posib...,795.0,https://cdn.wallapop.com/images/10420/e5/q2/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-09
3,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña barata y nueva,"Vendo boco nueva, nunca ha sido utilizado.",115.0,https://cdn.wallapop.com/images/10420/e4/sm/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-03
4,https://es.wallapop.com/item/bicicleta-nueva-8...,BICICLETA NUEVA,"Bicicleta nueva de hace un año, usada solo dos...",300.0,https://cdn.wallapop.com/images/10420/e3/ty/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-28
5,https://es.wallapop.com/item/bicicleta-canonda...,Bicicleta Cannondale,Bicicletas nuevas a estrenar marca Cannondale ...,550.0,https://cdn.wallapop.com/images/10420/e3/d1/__...,Bicicletas de carretera,Nuevo,False,NaN,M,2022-11-25
6,https://es.wallapop.com/item/bicicleta-de-grav...,Bicicleta de gravel NUEVA,"Nueva con garantia, todas las tallas",595.0,https://cdn.wallapop.com/images/10420/e2/2v/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-24
7,https://es.wallapop.com/item/casco-de-bici-o-p...,casco de bici o patín,Está nuevo lo compramos y nunca lo usamos,30.0,https://cdn.wallapop.com/images/10420/e1/vi/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-16
8,https://es.wallapop.com/item/bicicleta-adulto-...,bicicleta adulto,"Bicicleta adulto Decathlon. 3 platos, 7 marcha...",90.0,https://cdn.wallapop.com/images/10420/dv/no/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-08
9,https://es.wallapop.com/item/bicicleta-de-mont...,Bicicleta de montaña,"Hard rock specialiced Talla M casi nueva, la u...",300.0,https://cdn.wallapop.com/images/10420/dv/n7/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-26



Combination of **'Bicicletas plegables' & 'Nuevo'**

In [35]:
# Click the filter button on the website
click_filter()

# Function to select the Subfield
subfield_select()

time.sleep(2)

select_type('Bicicletas de carretera', 'Bicicletas plegables')

# Go back
back_filter_type()

# Apply all the filters
apply_final_filter()

In [36]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [37]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

44


In [38]:
# Web scarping from listing
df_2 = web_scarping(posts_num)

df_main = pd.concat([df_main, df_2], axis = 0)

In [39]:
df_main

,Link,Titles,Description,Price,Image,Type,State,Children,Size,Size_letter,Date
0,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña,Bicicleta de montaña para niño/a a partir de 1...,90.0,https://cdn.wallapop.com/images/10420/e5/qy/__...,Bicicletas de carretera,Nuevo,True,NaN,NaN,2022-12-08
1,https://es.wallapop.com/item/bicicleta-deporti...,bicicleta deportiva 290€,No la he usado esta nueva,290.0,https://cdn.wallapop.com/images/10420/e5/q0/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-08
2,https://es.wallapop.com/item/bicicleta-krn-kra...,Bicicleta KRN Krampus Monster Truck,Cuadro de Aluminio Horquilla de Aluminio Posib...,795.0,https://cdn.wallapop.com/images/10420/e5/q2/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-09
3,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña barata y nueva,"Vendo boco nueva, nunca ha sido utilizado.",115.0,https://cdn.wallapop.com/images/10420/e4/sm/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-03
4,https://es.wallapop.com/item/bicicleta-nueva-8...,BICICLETA NUEVA,"Bicicleta nueva de hace un año, usada solo dos...",300.0,https://cdn.wallapop.com/images/10420/e3/ty/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-28
...,...,...,...,...,...,...,...,...,...,...,...
39,https://es.wallapop.com/item/bicicleta-601207848,Bicicleta,"Bicicleta perfecto estado, solo por el defecto...",135.0,https://cdn.wallapop.com/images/10420/9x/xy/__...,Bicicletas plegables,Nuevo,False,NaN,NaN,2022-12-06
40,https://es.wallapop.com/item/bicicleta-plegabl...,Bicicleta plegable Zippy,Bicicleta de aluminio plegable y totalmente aj...,190.0,https://cdn.wallapop.com/images/10420/av/6z/__...,Bicicletas plegables,Nuevo,False,NaN,NaN,2022-11-18
41,https://es.wallapop.com/item/bicicleta-plegabl...,bicicleta plegable con doble suspensión,"Plegable, ciudad y montaña, cambio Shimano 5 v...",100.0,https://cdn.wallapop.com/images/10420/8w/xv/__...,Bicicletas plegables,Nuevo,False,NaN,NaN,2022-11-12
42,https://es.wallapop.com/item/olov-bike-plegabl...,Olov Bike plegable,Bicicleta plegable con cuadro de aluminio y ca...,699.0,https://cdn.wallapop.com/images/10420/b0/gz/__...,Bicicletas plegables,Nuevo,False,NaN,NaN,2022-11-22



Combination of **'MTB' & 'Nuevo'**

In [40]:
# Click the filter button on the website
click_filter()

# Function for Subfield
subfield_select()

time.sleep(2)

select_type('Bicicletas plegables', 'MTB')

# Go back
back_filter_type()

# Apply all the filters
apply_final_filter()

In [41]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [42]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

58


In [43]:
# Web scarping from listing
df_3 = web_scarping(posts_num)

df_main = pd.concat([df_main, df_3], axis = 0)


In [44]:
df_main

,Link,Titles,Description,Price,Image,Type,State,Children,Size,Size_letter,Date
0,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña,Bicicleta de montaña para niño/a a partir de 1...,90.0,https://cdn.wallapop.com/images/10420/e5/qy/__...,Bicicletas de carretera,Nuevo,True,NaN,NaN,2022-12-08
1,https://es.wallapop.com/item/bicicleta-deporti...,bicicleta deportiva 290€,No la he usado esta nueva,290.0,https://cdn.wallapop.com/images/10420/e5/q0/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-08
2,https://es.wallapop.com/item/bicicleta-krn-kra...,Bicicleta KRN Krampus Monster Truck,Cuadro de Aluminio Horquilla de Aluminio Posib...,795.0,https://cdn.wallapop.com/images/10420/e5/q2/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-09
3,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña barata y nueva,"Vendo boco nueva, nunca ha sido utilizado.",115.0,https://cdn.wallapop.com/images/10420/e4/sm/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-03
4,https://es.wallapop.com/item/bicicleta-nueva-8...,BICICLETA NUEVA,"Bicicleta nueva de hace un año, usada solo dos...",300.0,https://cdn.wallapop.com/images/10420/e3/ty/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-28
...,...,...,...,...,...,...,...,...,...,...,...
53,https://es.wallapop.com/item/bicicleta-mtb-mer...,Merida Big Nine 20 antracita 2021,¡¡LIQUIDACION DE TEMPORADA!! • Cuadro aluminio...,561.0,https://cdn.wallapop.com/images/10420/dd/cn/__...,MTB,Nuevo,False,NaN,NaN,2022-11-19
54,https://es.wallapop.com/item/merida-big-nine-1...,Merida Big Nine 15 TFS 2021,¡¡LIQUIDACION DE TEMPORADA!! • Cuadro aluminio...,524.0,https://cdn.wallapop.com/images/10420/dd/cj/__...,MTB,Nuevo,False,NaN,NaN,2022-11-19
55,https://es.wallapop.com/item/rockrider-st-520-...,Rockrider st 540 nueva con extras y garantia M,"Se vende bicicleta rockrider st540 talla M, us...",390.0,https://cdn.wallapop.com/images/10420/d0/ro/__...,MTB,Nuevo,False,NaN,"M,",2022-11-24
56,https://es.wallapop.com/item/merida-big-nine-1...,Merida Big Nine 15 20022,"Mtb Merida Big Nine 15, cuadro aluminio, horqu...",490.0,https://cdn.wallapop.com/images/10420/cp/nu/__...,MTB,Nuevo,False,NaN,NaN,2022-11-30


### Combination of **'MTB' & 'Como nuevo'**

In [45]:
# Click the filter button on the website
click_filter()

#Open the state filter tab
state_filter()

time.sleep(2)


select_type('Nuevo', 'Como nuevo')

# Go back
back_filter_state()

# Apply all the filters
apply_final_filter()

In [46]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [47]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

250


In [48]:
# Web scarping from listing
df_4 = web_scarping(posts_num)

df_main = pd.concat([df_main, df_4], axis = 0)

In [49]:
df_main

,Link,Titles,Description,Price,Image,Type,State,Children,Size,Size_letter,Date
0,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña,Bicicleta de montaña para niño/a a partir de 1...,90.0,https://cdn.wallapop.com/images/10420/e5/qy/__...,Bicicletas de carretera,Nuevo,True,NaN,NaN,2022-12-08
1,https://es.wallapop.com/item/bicicleta-deporti...,bicicleta deportiva 290€,No la he usado esta nueva,290.0,https://cdn.wallapop.com/images/10420/e5/q0/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-08
2,https://es.wallapop.com/item/bicicleta-krn-kra...,Bicicleta KRN Krampus Monster Truck,Cuadro de Aluminio Horquilla de Aluminio Posib...,795.0,https://cdn.wallapop.com/images/10420/e5/q2/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-09
3,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña barata y nueva,"Vendo boco nueva, nunca ha sido utilizado.",115.0,https://cdn.wallapop.com/images/10420/e4/sm/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-03
4,https://es.wallapop.com/item/bicicleta-nueva-8...,BICICLETA NUEVA,"Bicicleta nueva de hace un año, usada solo dos...",300.0,https://cdn.wallapop.com/images/10420/e3/ty/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-28
...,...,...,...,...,...,...,...,...,...,...,...
245,https://es.wallapop.com/item/bicicleta-merida-...,Bicicleta MÉRIDA en buen estado,"27 piñones, rueda 26. Talla S. Muy bien cuidada",160.0,https://cdn.wallapop.com/images/10420/cs/u8/__...,MTB,Como nuevo,False,NaN,NaN,2022-10-15
246,https://es.wallapop.com/item/bicicleta-btwin-m...,Bicicleta Btwin MTB 340,Bicicleta Btwin 340. Talla S/M,145.0,https://cdn.wallapop.com/images/10420/cq/e1/__...,MTB,Como nuevo,False,NaN,NaN,2022-11-01
247,https://es.wallapop.com/item/bicicleta-rockrid...,Bicicleta Rockrider 340,"Bicicleta nueva, solo utilizada dos veces. Tal...",140.0,https://cdn.wallapop.com/images/10420/ck/ut/__...,MTB,Como nuevo,False,NaN,NaN,2022-10-25
248,https://es.wallapop.com/item/bicicleta-29-mtb-...,"BICICLETA 29"" MTB MERIDA BIG NINE 900 T.XL.",Bicicleta de montaña Merida Big Nine 900 con r...,650.0,https://cdn.wallapop.com/images/10420/d4/al/__...,MTB,Como nuevo,False,21.0,NaN,2022-12-08


Combination of **'Bicicletas plegables' & 'Como nuevo'**

In [50]:
# Click the filter button on the website
click_filter()

# Function for Subfield
subfield_select()

time.sleep(2)

select_type('MTB', 'Bicicletas plegables')

# Go back
back_filter_type()

# Apply all the filters
apply_final_filter()

In [51]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [52]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

144


In [53]:
# Web scarping from listing
df_5 = web_scarping(posts_num)

df_main = pd.concat([df_main, df_5], axis = 0)

In [54]:
df_main

,Link,Titles,Description,Price,Image,Type,State,Children,Size,Size_letter,Date
0,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña,Bicicleta de montaña para niño/a a partir de 1...,90.0,https://cdn.wallapop.com/images/10420/e5/qy/__...,Bicicletas de carretera,Nuevo,True,NaN,NaN,2022-12-08
1,https://es.wallapop.com/item/bicicleta-deporti...,bicicleta deportiva 290€,No la he usado esta nueva,290.0,https://cdn.wallapop.com/images/10420/e5/q0/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-08
2,https://es.wallapop.com/item/bicicleta-krn-kra...,Bicicleta KRN Krampus Monster Truck,Cuadro de Aluminio Horquilla de Aluminio Posib...,795.0,https://cdn.wallapop.com/images/10420/e5/q2/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-09
3,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña barata y nueva,"Vendo boco nueva, nunca ha sido utilizado.",115.0,https://cdn.wallapop.com/images/10420/e4/sm/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-03
4,https://es.wallapop.com/item/bicicleta-nueva-8...,BICICLETA NUEVA,"Bicicleta nueva de hace un año, usada solo dos...",300.0,https://cdn.wallapop.com/images/10420/e3/ty/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-28
...,...,...,...,...,...,...,...,...,...,...,...
139,https://es.wallapop.com/item/monty-aluminium-f...,Monty Aluminium F18 LOGIK,"Monty Aluminium F18 LOGIK roja. Poco uso, está...",160.0,https://cdn.wallapop.com/images/10420/dy/f1/__...,Bicicletas plegables,Como nuevo,False,NaN,NaN,2022-12-02
140,https://es.wallapop.com/item/ultrasport-f-bike...,Ultrasport F-Bike,"Ultrasport F-Bike, Entrenador Bicicleta, Bicic...",80.0,https://cdn.wallapop.com/images/10420/dm/9h/__...,Bicicletas plegables,Como nuevo,False,NaN,NaN,2022-11-19
141,https://es.wallapop.com/item/tern-link-b7---te...,Tern Link B7 + Tern cargo rack,Bici plegable Tern Link B7 en muy buen estado ...,450.0,https://cdn.wallapop.com/images/10420/da/h3/__...,Bicicletas plegables,Como nuevo,False,NaN,NaN,2022-11-06
142,https://es.wallapop.com/item/dahon-plegable-74...,Dahon plegable,"Bicicleta plegable Dahon, muy buena calidad, r...",200.0,https://cdn.wallapop.com/images/10420/ca/uz/__...,Bicicletas plegables,Como nuevo,False,NaN,NaN,2022-10-24


Combination of **'Bicicletas de carretera' & 'Como nuevo'**

In [55]:
# Click the filter button on the website
click_filter()

# Function for Subfield
subfield_select()

time.sleep(2)

select_type('Bicicletas plegables', 'Bicicletas de carretera')

# Go back
back_filter_type()

# Apply all the filters
apply_final_filter()

In [56]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [57]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

167


In [58]:
# Web scarping from listing
df_6 = web_scarping(posts_num)

df_main = pd.concat([df_main, df_6], axis = 0)

In [59]:
df_main

,Link,Titles,Description,Price,Image,Type,State,Children,Size,Size_letter,Date
0,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña,Bicicleta de montaña para niño/a a partir de 1...,90.0,https://cdn.wallapop.com/images/10420/e5/qy/__...,Bicicletas de carretera,Nuevo,True,NaN,NaN,2022-12-08
1,https://es.wallapop.com/item/bicicleta-deporti...,bicicleta deportiva 290€,No la he usado esta nueva,290.0,https://cdn.wallapop.com/images/10420/e5/q0/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-08
2,https://es.wallapop.com/item/bicicleta-krn-kra...,Bicicleta KRN Krampus Monster Truck,Cuadro de Aluminio Horquilla de Aluminio Posib...,795.0,https://cdn.wallapop.com/images/10420/e5/q2/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-09
3,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña barata y nueva,"Vendo boco nueva, nunca ha sido utilizado.",115.0,https://cdn.wallapop.com/images/10420/e4/sm/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-03
4,https://es.wallapop.com/item/bicicleta-nueva-8...,BICICLETA NUEVA,"Bicicleta nueva de hace un año, usada solo dos...",300.0,https://cdn.wallapop.com/images/10420/e3/ty/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-28
...,...,...,...,...,...,...,...,...,...,...,...
162,https://es.wallapop.com/item/orbea-onix-2012-d...,Orbea Onix 2012 de carbono talla 57 grupo Ultegra,Componentes: - Cuadro tija y horquilla de carb...,800.0,https://cdn.wallapop.com/images/10420/dk/0x/__...,Bicicletas de carretera,Como nuevo,False,57.0,NaN,2022-11-29
163,https://es.wallapop.com/item/sillin-fizik-alia...,Sillin Fizik Aliante R7,"Sillín Fizik, con muy poco uso. Se compro con ...",40.0,https://cdn.wallapop.com/images/10420/bw/x3/__...,Bicicletas de carretera,Como nuevo,False,NaN,NaN,2022-10-23
164,https://es.wallapop.com/item/coluer-radar-al40...,COLUER RADAR AL 40,"Bicicleta de carretera con cuadro de aluminio,...",475.0,https://cdn.wallapop.com/images/10420/bt/bl/__...,Bicicletas de carretera,Como nuevo,False,NaN,NaN,2022-11-11
165,https://es.wallapop.com/item/pedales-lyotard-5...,Pedales Lyotard,Pedales bici clásica de carretera de la marca ...,20.0,https://cdn.wallapop.com/images/10420/9a/vx/__...,Bicicletas de carretera,Como nuevo,False,NaN,NaN,2022-10-30


Combination of **'Bicicletas de carretera' & 'En buen estado'**

In [60]:
# Click the filter button on the website
click_filter()

# Open the state filter tab
state_filter()

time.sleep(2)

select_type('Como nuevo', 'En buen estado')

# Go back
back_filter_state()

# Apply all the filters
apply_final_filter()

In [61]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [62]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

250


In [63]:
# Web scarping from listing
df_7 = web_scarping(posts_num)

df_main = pd.concat([df_main, df_7], axis = 0)

Combination of **'Bicicletas plegables' & 'En buen estado'**

In [64]:
# Click the filter button on the website
click_filter()

# Function for Subfield
subfield_select()

time.sleep(2)

select_type('Bicicletas de carretera', 'Bicicletas plegables')

# Go back
back_filter_type()

# Apply all the filters
apply_final_filter()

In [65]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [66]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

131


In [67]:
# Web scarping from listing
df_8 = web_scarping(posts_num)

df_main = pd.concat([df_main, df_8], axis = 0)


Combination of **'MTB' & 'En buen estado'**

In [68]:
# Click the filter button on the website
click_filter()

# Function for Subfield
subfield_select()

time.sleep(2)

select_type('Bicicletas plegables', 'MTB')

# Go back
back_filter_type()

# Apply all the filters
apply_final_filter()

In [69]:
#Scroll down to get all posts or 250
scroll_down()

#Scroll to the top of the page to start web scraping
scroll_to_top()

In [70]:
#Count the total available results and set the limit to 250 or all available results
posts_num = len(driver.find_elements(By.XPATH, '//div[@class="ItemCard__image ItemCard__image--with-description"]'))

if posts_num > 250:
    posts_num = 250
else:
    posts_num = posts_num
print(posts_num)

250


In [71]:
# Web scarping from listing
df_9 = web_scarping(posts_num)

df_main = pd.concat([df_main, df_9], axis = 0)

In [72]:
# Resetting index of Main DataFrame
df_main.reset_index(drop = True)

,Link,Titles,Description,Price,Image,Type,State,Children,Size,Size_letter,Date
0,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña,Bicicleta de montaña para niño/a a partir de 1...,90.0,https://cdn.wallapop.com/images/10420/e5/qy/__...,Bicicletas de carretera,Nuevo,True,NaN,NaN,2022-12-08
1,https://es.wallapop.com/item/bicicleta-deporti...,bicicleta deportiva 290€,No la he usado esta nueva,290.0,https://cdn.wallapop.com/images/10420/e5/q0/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-08
2,https://es.wallapop.com/item/bicicleta-krn-kra...,Bicicleta KRN Krampus Monster Truck,Cuadro de Aluminio Horquilla de Aluminio Posib...,795.0,https://cdn.wallapop.com/images/10420/e5/q2/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-09
3,https://es.wallapop.com/item/bicicleta-montana...,Bicicleta montaña barata y nueva,"Vendo boco nueva, nunca ha sido utilizado.",115.0,https://cdn.wallapop.com/images/10420/e4/sm/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-12-03
4,https://es.wallapop.com/item/bicicleta-nueva-8...,BICICLETA NUEVA,"Bicicleta nueva de hace un año, usada solo dos...",300.0,https://cdn.wallapop.com/images/10420/e3/ty/__...,Bicicletas de carretera,Nuevo,False,NaN,NaN,2022-11-28
...,...,...,...,...,...,...,...,...,...,...,...
1325,https://es.wallapop.com/item/bicicleta-gran-ca...,Bicicleta Gran Canyon MTB,Buen estado,500.0,https://cdn.wallapop.com/images/10420/df/kg/__...,MTB,Bueno,False,NaN,NaN,2022-10-15
1326,https://es.wallapop.com/item/bicicleta-797219180,Bicicleta,Vendo dos bicicletas antiguas a 40€ cada una. ...,40.0,https://cdn.wallapop.com/images/10420/d6/n6/__...,MTB,Bueno,False,NaN,NaN,2022-12-08
1327,https://es.wallapop.com/item/mountanbike-desce...,bicicleta fr6 evo descenso doble suspensión,Mountanbike descenso doble suspensión rockride...,300.0,https://cdn.wallapop.com/images/10420/dm/ak/__...,MTB,Bueno,False,NaN,NaN,2022-12-04
1328,https://es.wallapop.com/item/bicicleta-trek-se...,Bicicleta de montaña Trek Serie 6,Trek Serie 6 en perfectas condiciones Talla M/...,458.0,https://cdn.wallapop.com/images/10420/dh/o0/__...,MTB,Bueno,False,NaN,NaN,2022-11-29


In [76]:
# Converting columns into correct datatypes
df_main['Children'] = df_main['Children'].astype('bool')
df_main['Price'] = df_main['Price'].astype('float')
df_main['Size'] = df_main['Size'].astype('float')
df_main['Date'] =  pd.to_datetime(df_main['Date'], format='%d/%m/%Y')

In [77]:
# Dropping all the duplicates from the main data frame and adding the final values to df_final
df_final = df_main.drop_duplicates()

In [78]:
df_final.dtypes

Link                   object
Titles                 object
Description            object
Price                 float64
Image                  object
Type                   object
State                  object
Children                 bool
Size                  float64
Size_letter            object
Date           datetime64[ns]
dtype: object

In [79]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1330 entries, 0 to 249
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Link         1330 non-null   object        
 1   Titles       1330 non-null   object        
 2   Description  1330 non-null   object        
 3   Price        1330 non-null   float64       
 4   Image        1330 non-null   object        
 5   Type         1330 non-null   object        
 6   State        1330 non-null   object        
 7   Children     1330 non-null   bool          
 8   Size         36 non-null     float64       
 9   Size_letter  106 non-null    object        
 10  Date         1330 non-null   datetime64[ns]
dtypes: bool(1), datetime64[ns](1), float64(2), object(7)
memory usage: 115.6+ KB


Finally, in this step we are creating a new data frame aggregating the data for 'Type' and 'State columns by average prices.

In [80]:
result = df_final.groupby(['Type','State']).agg({'Price': ['mean']})
result.columns = ['price_mean']
agg = result.reset_index()

In [81]:
agg

,Type,State,price_mean
0,Bicicletas de carretera,Bueno,273.139560
1,Bicicletas de carretera,Como nuevo,329.766407
2,Bicicletas de carretera,Nuevo,317.805556
3,Bicicletas plegables,Bueno,113.518702
4,Bicicletas plegables,Como nuevo,168.354167
5,Bicicletas plegables,Nuevo,259.225455
6,MTB,Bueno,257.371960
7,MTB,Como nuevo,285.323960
8,MTB,Nuevo,405.372241
